# Importing libraries

In [1]:
import pandas as pd
import os
import csv
import datetime
import time
from pytz import timezone
import tweepy
import json

# Loading the credentials

In [2]:
# Load the Twitter API credentials from the config file
with open('config.json', 'r') as f:
    config = json.load(f)
    consumer_key = config['consumer_key']
    consumer_secret = config['consumer_secret']
    access_token = config['access_token']
    access_token_secret = config['access_token_secret']
    
# Verify the Twitter API credentials
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
try:
    api = tweepy.API(auth, wait_on_rate_limit=True)
    user = api.verify_credentials()
    print("Twitter API connection successful.")
except tweepy.error.TweepError as e:
    print("Error: Failed to verify Twitter API credentials.")
    print(e)

Twitter API connection successful.


# Disabling warnings

In [3]:
import warnings
warnings.filterwarnings("ignore")

# Downloading twitter data

In [11]:
# Define the topic and date range
topic = 'Ireland'
end_date = datetime.datetime(2023, 5, 21, tzinfo=timezone('Europe/London'))
start_date = datetime.datetime(2022, 5, 21, tzinfo=timezone('Europe/London'))

# Fetch tweets on the specified topic
try:
    tweets = []
    for tweet in tweepy.Cursor(api.search_tweets, q=topic, lang='en', tweet_mode='extended').items(2000):
        tweets.append({
            'Date': tweet.created_at.date(),
            'Tweet': tweet.full_text
        })
    print("Tweets downloaded successfully.")
except tweepy.TweepyException as e:
    if e.api_code == 88:
        # Rate limit reached, wait for the specified duration
        wait_time = int(e.response.headers['Retry-After'])
        print("Rate limit reached. Sleeping for:", wait_time, "seconds.")
        time.sleep(wait_time)
    print("Error: Failed to download tweets.")
    print(e)

Rate limit reached. Sleeping for: 122


Tweets downloaded successfully.


# Saving twitter data

In [13]:
# Convert the tweets list into a DataFrame
df_new = pd.DataFrame(tweets)

# Check if the CSV file already exists
if os.path.isfile('tweets.csv'):
    # Read the existing data from the CSV file
    df_existing = pd.read_csv('tweets.csv')
    
    # Concatenate the existing and new data
    df_combined = pd.concat([df_existing, df_new], ignore_index=True)
    
    # Save the combined DataFrame to the CSV file
    df_combined.to_csv('tweets.csv', index=False)
    print("Tweets appended to the existing CSV file.")
else:
    # Save the new DataFrame to a new CSV file
    df_new.to_csv('tweets.csv', index=False)
    print("New CSV file created with the downloaded tweets.")

Tweets appended to tweets.csv


# Sentiment analysis

In [14]:
from textblob import TextBlob
import pandas as pd

# Load the tweets from the CSV file
df = pd.read_csv('tweets.csv')

# Perform sentiment analysis using TextBlob
df['sentiment'] = df['Tweet'].apply(lambda x: TextBlob(x).sentiment.polarity)

# Classify sentiment as positive, negative, or neutral
df['sentiment_label'] = df['sentiment'].apply(lambda x: 'Positive' if x > 0 else 'Negative' if x < 0 else 'Neutral')

# Save the updated DataFrame to CSV
df.to_csv('tweets_sentiment.csv', index=False)

# Time series forecast of the sentiment

In [15]:
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio
from statsmodels.tsa.arima.model import ARIMA
from datetime import timedelta

# Load the tweets sentiment data from the CSV file
df = pd.read_csv('tweets_sentiment.csv')
df['Date'] = pd.to_datetime(df['Date'])

# Resample the data to daily frequency
df_daily = df.set_index('Date').resample('D').mean()

# Fit an ARIMA model to the sentiment data
model = ARIMA(df_daily['sentiment'], order=(1, 0, 0))
model_fit = model.fit()

# Generate predictions for the next 3 months
forecast_start_date = df_daily.index[-1] + timedelta(days=1)
forecast_end_date = forecast_start_date + timedelta(days=90)
forecast = model_fit.get_forecast(steps=(forecast_end_date - forecast_start_date).days)
forecast_mean = forecast.predicted_mean

# Create the Plotly figure for the sentiment forecast
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_daily.index, y=df_daily['sentiment'], name='Actual Sentiment'))
fig.add_trace(go.Scatter(x=forecast_mean.index, y=forecast_mean, name='Forecasted Sentiment'))

# Customize the layout of the figure
fig.update_layout(title='Sentiment Forecast',
                  xaxis_title='Date',
                  yaxis_title='Sentiment',
                  legend_title='Sentiment Type')

# Save the figure as an HTML file
pio.write_html(fig, file='sentiment_forecast.html', auto_open=True)

KeyError: 'Date'